In [1]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
import matplotlib.pyplot as plt
from scipy import stats

import sys; sys.path.insert(1, '/home/alessandro/thesis_material/EURECA_scripts')
import os

import geometry as gm
#import conversion as cnv
from func_statistics import compute_pdf_nbin, slopes_r_p, slopes_r_p_onlysub, slopes_r_p_mix, slope_time, slope_maps
import regression
from coare_functions import *
import meteo
import lcl
import metpy.calc as mpcalc
from metpy.units import units


import cartopy.crs as ccrs
crs = ccrs.PlateCarree()
#from plotmap import plot_background
from plotdistr import * #fb_distribution_npoint, perc_distribution, scatterplot_fit, hist2d
from pdist_fb_distrib_save_fast import *

import warnings; warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# find data
path = '/saturn/simu_Caribbean/coupled_2km/output/'
files_ls = ['QCLOUD', 'U', 'V', 'W', 'OLR', 'PH', 'PHB', 'QVAPOR', 'PB', 'P', 'T']
files_ls = [name + '.nc' for name in files_ls]
wrf_files = [path+name for name in files_ls]
#fname_list = sorted(glob.glob(wrf_files + '*'))

In [4]:
# import sea_mask
if os.path.exists('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_new.npy'):
    sea_mask = np.load('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_new.npy')
    
if os.path.exists('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_noWarmPatch_noWest.npy'):
    sea_mask_modified = np.load('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_noWarmPatch_noWest.npy')
    sea_mask_modified_nan = np.ones_like(sea_mask_modified)
    sea_mask_modified_nan[sea_mask_modified==0] = np.nan

fourier_mask = np.load('/home/alessandro/thesis_material/EURECA_scripts/masks/fourier_large.npy')
    
tourbillons_mask = np.load('/home/alessandro/thesis_material/EURECA_scripts/masks/tourbillons_mask.npy')
open_ocean_mask = np.load('/home/alessandro/thesis_material/EURECA_scripts/masks/open_ocean_mask.npy')

In [5]:
# MATTE AVEVA GIA' CALCOLATO TUTTE LE CORRELAZIONI 
# E HA STIMATO QUESTA LUNGHEZZA?
L = 30
nskip = int(L/2)
nt = int(1)

# from top pbl upwards
# SOPRA PBL LA LUNGEHZZA DI CORRELAZIONE AUMENTA?
# NEL SENSO CHE I SEGNALI HANNO PIù SPAZIO PER PROPAGARSI?
Ltop = 150
nskiptop = int(Ltop/2)
nttop = int(1)

# reference population mean
popmean = 0.

In [6]:
wrf_hourly = path + 'Caraibes_WRF_3h_surf.nc'
ds_hourly = xr.open_dataset(wrf_hourly)
ds_hourly = ds_hourly.sel(time_counter=slice('2020-02-01T01:30:00.000000000', '2020-02-29T22:30:00.000000000'))

# lat and lon
lon = ds_hourly['nav_lon'].values
lat = ds_hourly['nav_lat'].values

In [7]:
U, V = ds_hourly['U_PHYL1']*sea_mask_modified_nan, ds_hourly['V_PHYL1']*sea_mask_modified_nan
U = np.sqrt(U.values**2 + V.values**2)

In [8]:
T2 = ds_hourly['T2'].values*sea_mask_modified_nan-273.15

In [9]:
###   RETRIEVE PRESSURE   -  MOSTLY FOR PLOTTING

pres_base = xr.open_dataset(wrf_files[files_ls.index('PB.nc')])
pres_base = pres_base['PB']

pres = xr.open_dataset(wrf_files[files_ls.index('P.nc')])
pres = pres['P'].sel(time_counter=slice('2020-02-01T01:30:00.000000000', '2020-03-01T01:30:00.000000000'))


In [10]:
pres = ((pres+pres_base)/100.).values*sea_mask_modified_nan
pres_sfc = pres[:,0,:,:]
del pres_base, pres

In [11]:
mean_pres_sfc = np.nanmean(pres_sfc, axis=0)
sfc_pres = np.zeros_like(T2)
sfc_pres[:] = mean_pres_sfc


In [12]:
q2 = ds_hourly['Q2'].values*sea_mask_modified_nan

RH = q2/meteo.qsea(T2, sfc_pres)

del q2

In [13]:
sst = ds_hourly['SST'].values*sea_mask_modified_nan - 273.15; 
PBLH = ds_hourly['PBLH'].values*sea_mask_modified_nan; 

In [15]:
SWDOWN, GLW = ds_hourly['GSW'].values*sea_mask_modified_nan, ds_hourly['GLW'].values*sea_mask_modified_nan

In [ ]:
# u = ocean surface relative wind speed (m/s) at height zu(m)
# t = bulk air temperature (degC) at height zt(m)
# rh = relative humidity (%) at height zq(m)
# ts = sea water temperature (degC) - see jcool below
# P = surface air pressure (mb) (default = 1015)
# Rs = downward shortwave radiation (W/m^2) (default = 150)
# Rl = downward longwave radiation (W/m^2) (default = 370)
# zu = wind sensor height (m) (default = 18m)
# zt = bulk temperature sensor height (m) (default = 18m)
# zq = RH sensor height (m) (default = 18m)
# lat = latitude (default = 45 N)
# zi = PBL height (m) (default = 600m)
# rain = rain rate (mm/hr)
# cp = phase speed of dominant waves (m/s)
# sigH =  significant wave height (m)
# jcool = cool skin option (default = 1 for bulk SST)

jcool=1
u_star, tau, HFX_coare, LHF_coare, C_u, C_h, C_lh, L = coare_var(U, T2, RH, sst, sfc_pres, SWDOWN, GLW, lat, PBLH, jcool)